In [ ]:
import pandas as pd
from loader import *
import matplotlib.pyplot as plt


In [ ]:
import jieba
import jieba.analyse


In [ ]:
import torch 

In [ ]:
a = torch.rand((1,2,3))
b = torch.rand((1,3,3))
a,b

In [ ]:
torch.cat((a, b), dim=1)

In [ ]:
with open('./data/stop_word.txt') as f:
    text = f.read()
    text = text.split('\n')

In [ ]:
[1,2] + [2,3]

In [ ]:
tags = jieba.analyse.extract_tags('醫師：我們6月30號抽血，免疫細胞指數還不錯這樣子，然後病毒量也是OK，阿血糖，糖化血色素7.2，還不錯，所以控制的還不錯，所以表示這段時間身體狀況應該還好。民眾：因為我有重訓和訓練……醫師：低密度膽固醇101是正常，是可以的，然後因為有一點點尿酸稍微高一點點，但是還不需要吃藥，就是飲食稍微注意一下就好，會造成尿酸高一點的食物稍微控制一下就好。民眾：我不知道哪些。醫師：不')
tags

In [ ]:
a = torch.tensor([
    [
        [1,2,3],
        [0,0,0],
        [3,4,3]
    ],
    [
        [1,2,3],
        [1,2,3],
        [3,4,3]
    ],
], dtype=torch.float)
b = torch.tensor([
    [
        [1,2,3],
        [3,4,3]
    ],
    [
        [1,2,3],
        [3,4,3]
    ],
], dtype=torch.float)
cosine_sim()(a, b)

In [ ]:
cosine_sim()(a, b)

In [ ]:
torch.max(cosine_sim()(a, b), 1)

In [ ]:
# output = torch.randn(2, 4, 768)
# choi_output = torch.randn(2, 3, 768)
cosine_sim(None)(a, b)

In [ ]:
vec1=a
vec2=b

vec2 = torch.transpose(vec2, 1, 2)
norm1 = torch.linalg.norm(vec1, dim=2)
norm2 = torch.linalg.norm(vec2, dim=1)   
norm1 = norm1.unsqueeze(-1).expand(2, vec1.size(1), vec1.size(2))
norm2 = norm2.unsqueeze(2).expand(2, vec2.size(2), vec2.size(1))
norm2 = torch.transpose(norm2, 1, 2)
eledot = torch.matmul(torch.div(vec1, norm1), torch.div(vec2, norm2))

# print(torch.mul(norm1.transpose(0, 1), norm2))
# cos_sim = torch.div(eledot ,torch.matmul(norm1.transpose(0, 1), norm2))
eledot

In [ ]:
norm2 = norm2.unsqueeze(2).expand(2, vec2.size(2), vec2.size(1))
norm2 = torch.transpose(norm2, 1, 2)

In [ ]:
' rock '

In [ ]:
from loader import QA_Dataset
from transformers import AutoTokenizer
config = 'hfl/chinese-xlnet-base'
tokenizer = AutoTokenizer.from_pretrained(config)


In [ ]:
ins = QA_Dataset(tokenizer, './data/train_fold1.json').__getitem__(1)
ins['q_input_ids']

In [ ]:
tokenizer.decode(ins['q_input_ids'][0])

In [ ]:
import pandas as pd
from loader import *
import matplotlib.pyplot as plt
from transformers import AutoTokenizer


config = 'hfl/chinese-xlnet-base'
tokenizer = AutoTokenizer.from_pretrained(config)
doc, q, cho, ans = json_parser()

In [ ]:
pipe =  doc_preprocessing(config)
d = pipe(doc, q)

# ori_len = [len(window) for frag in d for window in frag['input_ids']]
# plt.hist(ori_len, color = 'black', cumulative = False)
# plt.show()

In [ ]:
tokenizer.decode(d[0]['q_input_ids'][0])

In [ ]:
plt.hist(strQ2B(ans))
plt.show()


## LOADER

In [ ]:
from torch.utils.data import DataLoader
from loader import QA_Dataset
from torch.nn.utils.rnn import pad_sequence
import torch

config = 'hfl/chinese-xlnet-base'
lr = 1e-3

In [ ]:
import  torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
a = torch.ones(25, 300)
b = torch.ones(22, 300)
c = torch.ones(15, 300)
pad_sequence([a, b, c], batch_first=True,).shape

In [ ]:
def fragment_pad(*dcts):
    combined_dict = {}
    for k in set(dcts[0]):
        if not k=='label' and not k=='choice':
            print(k, [d[k].shape for d in dcts])
            combined_dict.update({
                k:pad_sequence([d[k].squeeze(0) for d in dcts], batch_first=True)
            })
        if k=='label':
             combined_dict.update({
                'label':torch.stack([d['label'] for d in dcts]) 
             })
    return combined_dict

def collate_fn(batch):
    batched_dict = fragment_pad(*batch)
    return batched_dict

train_ds = QA_Dataset(config, './data/Train_qa_ans.json')
train_loader = DataLoader(
    train_ds, 
    batch_size=4, 
    num_workers=1, 
    shuffle=True, 
    collate_fn=collate_fn
)
train_loader.__iter__().__next__()
# model = QA_Model(config, lr=lr)

In [ ]:
from loader import QA_Dataset

In [ ]:
from transformers import XLNetTokenizer, XLNetModel
import torch

config = 'hfl/chinese-xlnet-base'
tokenizer = XLNetTokenizer.from_pretrained(config)
tokenizer.add_special_tokens({'additional_special_tokens':['<q>']})
model = XLNetModel.from_pretrained(config)
model.resize_token_embeddings(len(tokenizer))

In [ ]:
c ='預計的性行為，前面兩個小時吃兩顆，24小時後，再吃一顆，再一個24小時之後吃一顆'

ids = tokenizer([c, c[0:30]], max_length=40, padding='max_length').input_ids
tokenizer.decode(ids[0])

In [ ]:
for ele in ids[0]:
    print(    tokenizer.decode(ele))


In [ ]:
tokenizer.decode(ids[1])

## Model

In [ ]:
h = model(**encoding).last_hidden_state

In [ ]:
from transformers.modeling_utils import SequenceSummary


In [ ]:
s = SequenceSummary(config)
getattr(config, 'summary_type', 'last')

In [48]:
import torch

a = torch.randn(2,3,7)
a


tensor([[[-1.4974, -1.8334, -0.0170,  1.3927, -0.3833, -2.2319,  0.2078],
         [-0.0762,  0.3411,  0.5927,  0.7143, -0.5335, -1.1608, -1.4399],
         [ 0.0720,  1.3276, -1.7206, -2.0686, -0.4708, -1.6096, -0.9726]],

        [[-0.4978,  0.6420,  0.3995,  1.3595, -1.1336,  0.8520, -1.5059],
         [ 0.0434, -1.8415, -0.9771,  1.5386, -0.2875,  0.8286, -0.3059],
         [ 0.3787, -0.6196,  0.0909, -0.6270, -1.9673,  0.7819, -1.1469]]])

In [60]:
index = torch.tensor([
    [ 1,2,1 ],
    [ 1,2,2 ],
])

max_val = [a.index_select(1, ele[1])[ele[0]] for ele in enumerate(index)]
# a.index_select(1, index)
max_val

[tensor([[-0.0762,  0.3411,  0.5927,  0.7143, -0.5335, -1.1608, -1.4399],
         [ 0.0720,  1.3276, -1.7206, -2.0686, -0.4708, -1.6096, -0.9726],
         [-0.0762,  0.3411,  0.5927,  0.7143, -0.5335, -1.1608, -1.4399]]),
 tensor([[ 0.0434, -1.8415, -0.9771,  1.5386, -0.2875,  0.8286, -0.3059],
         [ 0.3787, -0.6196,  0.0909, -0.6270, -1.9673,  0.7819, -1.1469],
         [ 0.3787, -0.6196,  0.0909, -0.6270, -1.9673,  0.7819, -1.1469]])]

In [65]:
 torch.cat(max_val).view(2,-1, 7)

tensor([[[-0.0762,  0.3411,  0.5927,  0.7143, -0.5335, -1.1608, -1.4399],
         [ 0.0720,  1.3276, -1.7206, -2.0686, -0.4708, -1.6096, -0.9726],
         [-0.0762,  0.3411,  0.5927,  0.7143, -0.5335, -1.1608, -1.4399]],

        [[ 0.0434, -1.8415, -0.9771,  1.5386, -0.2875,  0.8286, -0.3059],
         [ 0.3787, -0.6196,  0.0909, -0.6270, -1.9673,  0.7819, -1.1469],
         [ 0.3787, -0.6196,  0.0909, -0.6270, -1.9673,  0.7819, -1.1469]]])